# Example - Opportunity challenge.


#### Download and preprocess data

First, download the dataset for the opportunity challenge and set up the directories to hold the raw and processed data:

In [1]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
! mkdir -p data/raw
! mv OpportunityUCIDataset.zip data/raw

--2021-09-28 12:35:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306636009 (292M) [application/x-httpd-php]
Saving to: ‘OpportunityUCIDataset.zip’

OpportunityUCIDatas 100%[===================>] 292.43M  9.14MB/s    in 51s     

2021-09-28 12:36:44 (5.77 MB/s) - ‘OpportunityUCIDataset.zip’ saved [306636009/306636009]



Then, run the following script to preprocess the data. We collate all the training data into one file, same for the validation and testing data.

In [2]:
! python3 preprocess_opportunity.py

Checking dataset data/raw/OpportunityUCIDataset.zip
Processing dataset files ...
Generating training files
... file OpportunityUCIDataset/dataset/S1-Drill.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL1.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL2.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL3.dat -> train_data
... file OpportunityUCIDataset/dataset/S1-ADL4.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-Drill.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-ADL1.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-ADL2.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-Drill.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-ADL1.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-ADL2.dat -> train_data
... file OpportunityUCIDataset/dataset/S2-ADL3.dat -> train_data
... file OpportunityUCIDataset/dataset/S3-ADL3.dat -> train_data
Generating validation files
... file Opportun

#### Dataset Config

We choose the parameters of our sliding window, and specify the name and location of the target dataset.

In [3]:
opp_class_names = ['Null', 'Open Door 1', 'Open Door 2', 'Close Door 1', 'Close Door 2', 'Open Fridge',
                       'Close Fridge', 'Open Dishwasher', 'Close Dishwasher', 'Open Drawer 1', 'Close Drawer 1',
                       'Open Drawer 2', 'Close Drawer 2',
                       'Open Drawer 3', 'Close Drawer 3', 'Clean Table', 'Drink from Cup', 'Toggle Switch']

target_dataset = 'opportunity'
window_size = 24
window_step = 12
n_classes = 18

config_dataset = {
        "dataset": target_dataset,
        "window": window_size,
        "stride": window_step,
        "stride_test": 1,
        "path_processed": f"data/{target_dataset}",
    }

Import the data for training and validation (see datasets.py for sensor dataset implementation). We get the number of channels from the dataset.

In [4]:
from datasets import SensorDataset
dataset = SensorDataset(**config_dataset, prefix="train")
dataset_val = SensorDataset(**config_dataset, prefix="val")
n_channels = dataset.n_channels


Creating opportunity train HAR dataset of size 43985 ...
Creating opportunity val HAR dataset of size 2509 ...


### Import DeepConvLSTM class

See DeepConvLSTM_py3.py for implementation.

In [5]:
from DeepConvLSTM_py3 import DeepConvLSTM

#### Create an instance of DeepConvLSTM, with the number of channels and classes defined earlier. 
The dataset arg determines where the results and training checkpoints should be saved.


In [6]:
deepconv = DeepConvLSTM(n_channels=n_channels, n_classes=n_classes, dataset=target_dataset).cuda()

### Train the model


We'll train for only 30 epochs here as a demonstration. We use a learning rate scheduler to decrease the maximum learning rate for all parameters every 100 epochs.

In [7]:
from DeepConvLSTM_py3 import model_train

# Define train config options
config_train = {'batch_size': 256,
                'optimizer': 'Adam',
                'lr': 1e-3,
                'lr_step': 100,
                'lr_decay': 0.1,
                'init_weights': 'orthogonal',
                'epochs': 30,
                'clip_grad': 0,
                'print_freq': 100,
                'num_batches': 212,
                'num_batches_eval': 212}

model_train(deepconv, dataset, dataset_val, config_train, verbose=True)

Running HAR training loop ...
[-] Initializing weights (orthogonal)...
----------------------------------------------------------------------------------------------------
[-] Learning rate:  0.001
[-] Epoch 0/30	Train loss: 1.03 	acc: 72.24(%)	fm: 9.18(%)	fw: 64.31(%)	inf:0
[-] Epoch 0/30	Val loss: 0.66 	acc: 83.62(%)	fm: 5.12(%)	fw: 77.42(%)	inf:0
[*] Saving checkpoint... (0.0->0.05116573992781191)
----------------------------------------------------------------------------------------------------
[-] Learning rate:  0.001
[-] Epoch 1/30	Train loss: 0.74 	acc: 76.68(%)	fm: 22.13(%)	fw: 72.53(%)	inf:0
[-] Epoch 1/30	Val loss: 0.52 	acc: 83.98(%)	fm: 18.83(%)	fw: 82.15(%)	inf:0
[*] Saving checkpoint... (0.05116573992781191->0.18828245296798587)
----------------------------------------------------------------------------------------------------
[-] Learning rate:  0.001
[-] Epoch 2/30	Train loss: 0.63 	acc: 78.49(%)	fm: 27.99(%)	fw: 74.37(%)	inf:0
[-] Epoch 2/30	Val loss: 0.55 	acc: 83.

Load the testing data to evaluate the trained model, and setup the test configuration.

In [8]:
dataset_test = SensorDataset(**config_dataset, prefix="test")
test_config = {'batch_size': 256,
              'train_mode': False,
              'dataset': target_dataset,
              'num_batches_eval': 212}

Creating opportunity test HAR dataset of size 9894 ...


Import model_eval and test the model.

In [9]:
from DeepConvLSTM_py3 import model_eval
model_eval(deepconv, dataset_test, test_config, return_results=False)

Running HAR evaluation loop ...
[-] Loading checkpoint ...
[-] Test loss: 0.40	acc: 87.71(%)	fm: 54.94(%)	fw: 87.87(%)
[Finished HAR evaluation loop (h:m:s): 0:00:01


We can also use [thop](https://pypi.org/project/thop/) to count the number of floating point operations performed during a forward pass of one batch of synthetic data. We divide by the batch size to get the number of operations per window.

In [10]:
# Get number of flops
from thop import profile
import torch
deepconv = deepconv.train()

x = torch.ones([config_train['batch_size'], config_dataset['window'], n_channels]).cuda()
macs, params = profile(deepconv, inputs=(x,), verbose=True)
flops = macs / config_train['batch_size']
print(f'Number of floating point operations in a forward pass of one batch: {flops}')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_lstm() for <class 'torch.nn.modules.rnn.LSTM'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'DeepConvLSTM_py3.DeepConvLSTM'>. Treat it as zero Macs and zero Params.
Number of floating point operations in a forward pass of one batch: 115671040.0
